In [ ]:
import matplotlib.pyplot as plt
import hyperspy.api as hs
import numpy as np
import py4DSTEM
from py4DSTEM.visualize import show
print(py4DSTEM.__file__)
print('py4dstem : ', py4DSTEM.__version__)
print('Hyperspy : ', hs.__version__)
import h5py
import json

In [ ]:
file_path='template'

In [ ]:
# Do not delete this cell

## Prepare data

In [ ]:
## Get data
meta_path = file_path.split('_data')[-2]+ '.hdf'
with h5py.File(meta_path, 'r') as f:
    print(f['metadata'].keys())
    print('kV : ' , f['metadata/ht_value(V)'][()])
    nominal_CL = f['metadata/nominal_camera_length(m)'][()]
    print('camera_length : ', nominal_CL)
    
    HT = f['metadata/ht_value(V)'][()].astype(int)
    cam_len = (f['metadata/nominal_camera_length(m)'][()] * 100).astype(int) #CL in cm

In [ ]:
data_file =  file_path.split('_data')[-2] +'_data.hdf5'
f = h5py.File(data_file,'r')
print(f)
d = f['Experiments']['__unnamed__']['data'][:]
f.close()
print(data_file)
d = hs.signals.Signal2D(d)

In [ ]:
#reduce size in real space
d_bin4 = d.inav[:252, :252].rebin(scale = (4, 4, 1, 1))

In [ ]:
#pass to py4DSTEM
datacube = py4DSTEM.datacube.DataCube(data = d_bin4.data)

In [ ]:
datacube

In [ ]:
#add some approx calibration data
if HT == 300000 and cam_len == 20:
    pixel_size_inv_Ang = 0.0093
elif HT == 300000 and cam_len == 40:
    pixel_size_inv_Ang = 0.0049
elif HT == 300000 and cam_len == 80:
    pixel_size_inv_Ang = 0.0024
elif HT == 200000 and cam_len == 9:
    pixel_size_inv_Ang = 0.011
elif HT == 200000 and cam_len == 20:
    pixel_size_inv_Ang = 0.0053
elif HT == 200000 and cam_len == 40:
    pixel_size_inv_Ang = 0.0027
elif HT == 200000 and cam_len == 80:
    pixel_size_inv_Ang = 0.0014   
elif HT == 200000 and cam_len == 100:
    pixel_size_inv_Ang = 0.0009

pixel_size_inv_Ang = pixel_size_inv_Ang * float(eval(pixel_size_factor))

#Diffraction space
datacube.calibration.set_Q_pixel_size(pixel_size_inv_Ang)
datacube.calibration.set_Q_pixel_units('A^-1')

In [ ]:
pixel_size_inv_Ang

In [ ]:
datacube

In [ ]:
#remove hot pixels
datacube, mask = datacube.filter_hot_pixels(thresh = 0.1, return_mask=True)

In [ ]:
plt.figure()
plt.imshow(mask)

In [ ]:
#calculate the max and mean diff patterns from the full data set
datacube.get_dp_max();
datacube.get_dp_mean();

In [ ]:
#These are saved within the datacube 'tree' structure, we can see this by:
datacube.treekeys

In [ ]:
#now let's plot these
py4DSTEM.show([
        datacube.tree('dp_mean'), 
        datacube.tree('dp_max'), 
    ],
    cmap='inferno',
    power = 0.5,
)

In [ ]:
dp = datacube[10,30]
show(dp)

In [ ]:
probe_semiangle, probe_qx0, probe_qy0 = py4DSTEM.process.calibration.get_probe_size(
    dp,
)
print(probe_semiangle, probe_qx0, probe_qy0 )

In [ ]:
# Create an annular dark field (ADF) virtual detector using user-chosen values:
center =probe_qx0,probe_qy0# (dataset.shape[-1]//2,dataset.shape[-1]//2)
radii = (50,255)

# Plot the ADF detector
py4DSTEM.visualize.show(
    datacube.tree('dp_max'), 
    scaling='log',
    
    annulus = {
      'center':center,
      'radii':radii,
      'alpha':0.3,
      'fill':True
    }
)

# Calculate the ADF image
datacube.get_virtual_image(
    mode = 'annulus',
    geometry = ((center),radii),
    name = 'dark_field',
)

# Plot the ADF image
py4DSTEM.visualize.show(
    datacube.tree('dark_field'),
)

In [ ]:
datacube.treekeys

In [ ]:
# Try making a synthetic probe
syn_probe_width='3'
syn_probe_rad = int(probe_semiangle)
syn_probe_width = float(syn_probe_width)

syn_probe = py4DSTEM.braggvectors.probe.Probe.generate_synthetic_probe(syn_probe_rad, syn_probe_width, (datacube.data.shape[-1], datacube.data.shape[-1]))

In [ ]:
# Construct a probe template to use as a kernel for correlation disk detection
probe_semiangle = syn_probe_rad
syn_probe_kernel = syn_probe.get_kernel(
    mode = 'sigmoid',
    radii = (probe_semiangle * 1, probe_semiangle * 3.0),
    bilinear=True,
)

# Plot the probe kernel
py4DSTEM.visualize.show_kernel(
    syn_probe.kernel, 
    R=20, 
    L=20, 
    W=1,
    figsize = (8,4),
)


In [ ]:
#pick random positions with high intensity from df image
n_pos = 6 # number of positinos
df_mean =datacube.tree('dark_field').data.mean()
pos = np.where(datacube.tree('dark_field').data > df_mean)
xy_pos = np.zeros(shape = (2, n_pos))
for i in range(n_pos):
    rand_int = np.random.randint(0, pos[0].shape[0])
    xy_pos[0,i] = pos[0][rand_int]
    xy_pos[1,i] = pos[1][rand_int]

In [ ]:
# Choose some diffraction patterns to use for hyperparameter tuning

rxs = tuple(xy_pos[0].astype(int))
rys = tuple(xy_pos[1].astype(int))

py4DSTEM.visualize.show_points(
    datacube.tree('dark_field'),
    x=xy_pos[0],
    y=xy_pos[1],
    figsize=(8,8)
)

In [ ]:
# Test hyperparameters on a few probe positions
# Visualize the diffraction patterns and the located disk positions

# Hyperparameters
detect_params = {
    'corrPower': 1.0, #1.0,
    'sigma': 0,
    'edgeBoundary': 2,
    'minRelativeIntensity': 0,
    'minAbsoluteIntensity': 0.25, #0.5,
    'minPeakSpacing': 2,
    'subpixel' : 'poly',
#     'subpixel' : 'multicorr',
    'upsample_factor': 2,
    'maxNumPeaks': 1000,
#     'CUDA': True,
}

disks_selected = datacube.find_Bragg_disks(
    data = (rxs, rys),
    template = syn_probe_kernel,
    **detect_params,
)

py4DSTEM.visualize.show_image_grid(
    get_ar = lambda i:datacube.data[rxs[i],rys[i],:,:],
    H=2, 
    W=2,
    axsize=(5,5),
    vmax=1,
    power=0.1,
    get_x = lambda i: disks_selected[i].data['qx'],
    get_y = lambda i: disks_selected[i].data['qy'],
    open_circles = True,
    scale = 500,
)

In [ ]:
#now fit to all data
bragg_peaks = datacube.find_Bragg_disks(
    template = syn_probe_kernel,
    **detect_params,
)

In [ ]:
file_name_braggdisks_raw =file_path.split('_data')[-2] + '_CL_' +str(cam_len) + 'cm_bragg.h5'
print(file_name_braggdisks_raw)

In [ ]:
# Save Bragg disk positions
py4DSTEM.save(
    file_name_braggdisks_raw,
    bragg_peaks,
    mode='o',
)

In [ ]:
#have a look at the calibration state of the bragg vectors
bragg_peaks.calstate

In [ ]:
#print current calibrations
bragg_peaks.calibration

In [ ]:
#display one diff pattern with fit bragg peak postions
dp = datacube[8,30]
v = bragg_peaks.raw[8,30]

show(
    dp,
    power=0.1,
    vmax=0.0001,
    points = {
        'x' : v.qx,
        'y' : v.qy,
    }
)

In [ ]:
# compute the bragg vector map

bvm = bragg_peaks.histogram(
    mode='raw',
    sampling = 1,
)

show(bvm)

In [ ]:
# Measure the origin
center_guess = (probe_qx0,probe_qy0)
qx0_meas,qy0_meas,mask_meas = bragg_peaks.measure_origin(
    center_guess=center_guess,
    score_method='intensity',
)

fig, ax = show(
    [qx0_meas,qy0_meas],
    cmap = 'RdBu',
    returnfig=True,
)

In [ ]:
# Fit a plane to the origins
qx0_fit,qy0_fit,qx0_residuals,qy0_residuals = bragg_peaks.fit_origin(
    robust= True,
    robust_thresh= 1.2,
)

In [ ]:
# Now that we've calibrated the center positions, we can re-compute
# the Bragg vector map, this time with the center correction applied

sampling = 1

# compute
bvm = bragg_peaks.histogram(
    #mode='cal',             # 'cal' is the default mode, so this line can be included or left out
    sampling = sampling,
)

fig, ax = show(
    bvm,
    circle={
        'center' : bvm.origin,   # the centered BVM knows where its origin is 
        'R' : 4*sampling,
        'fill' : False,
        'linewidth' : 1
    },
    returnfig = True,
    vmax=0.999
)

In [ ]:
# Compare this to the uncalibrated BVM - much better!
# compute raw vs. centered
bvm_r = bragg_peaks.histogram( mode='raw', sampling=sampling )
bvm_c = bragg_peaks.histogram( mode='cal', sampling=sampling )

show( [bvm_r, bvm_c] ,vmax=0.9999, scaling = 'log')

L = 20
x,y = bvm_c.origin
x0,xf = np.round([x-L,x+L]).astype(int)
y0,yf = np.round([y-L,y+L]).astype(int)

show(
    [
    bvm_r[x0:xf,y0:yf],
    bvm_c[x0:xf,y0:yf]
    ],
    vmax=0.1,
    scaling = 'log'
)

In [ ]:
# Define gold structure 

# set lattice parameter and Z-number
a_lat = 4.08
atom_num = 79

# set max scattering angle, in inverse Angstroms
k_max = 1.5


# Define lattice
pos = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])

# Make crystal
crystal = py4DSTEM.process.diffraction.Crystal(
    pos, 
    atom_num, 
    a_lat)

crystal.calculate_structure_factors(k_max)
crystal.plot_scattering_intensity()

In [ ]:
# calibrate
bragg_peaks.calibration.set_Q_pixel_size(pixel_size_inv_Ang)
bragg_peaks.calibration.set_Q_pixel_units('A^-1')
bragg_peaks.setcal()

crystal.plot_scattering_intensity(
    bragg_peaks = bragg_peaks,
    bragg_k_power = 2.0
)

In [ ]:
# fit pixel size to lattice
crystal.calibrate_pixel_size(
    bragg_peaks = bragg_peaks,
    bragg_k_power = 4.0,
    plot_result = True,
    set_calibration_in_place = True
)

In [ ]:
# Select an annular region in which to perform a fit
# The ideal is a single, isolated ring of peaks -
#attempt at automating

pixel_size_inv_Ang = bragg_peaks.calibration['Q_pixel_size']

Au_111 = 2.35 # d-spacing, Angstrom
Au_200 = 2.04 # d-spacing, Angstrom
Au_220 = 1.44 # d-spacing, Angstrom
Au_311 = 1.23 # d-spacing, Angstrom

q_mid_px = sampling * int((1/ Au_220) / pixel_size_inv_Ang)
print(q_mid_px)
q_range = (q_mid_px - int(eval(ring_det_range)), q_mid_px + int(eval(ring_det_range)))
#if 220 too close to edge use 111 instead
if q_mid_px > 220*sampling:
    q_mid_px = sampling * int((1/ Au_111) / pixel_size_inv_Ang)
    q_range = (q_mid_px - int(eval(ring_det_range)), q_mid_px + int(eval(ring_det_range)))
    
if HT == 300000 and cam_len == 20:
    q_mid_px = sampling * int((1/ Au_311) / pixel_size_inv_Ang)
    print(q_mid_px)
    q_range = (q_mid_px - int(eval(ring_det_range)), q_mid_px + int(eval(ring_det_range)))
    

py4DSTEM.show(
    bvm_c,
    cmap='gray',
    intensity_range='absolute',
    vmin=0,
    vmax=10.0,
    annulus={
        'center':bvm_c.origin,
        'radii': q_range,'fill':True,'color':'r','alpha':0.3}
)

In [ ]:
# Fit the elliptical distortions
p_ellipse = py4DSTEM.process.calibration.fit_ellipse_1D(
    bvm_c,
    center = bvm_c.origin,
    fitradii = q_range,
)

# plot the fit
py4DSTEM.visualize.show_elliptical_fit(
    bvm_c,
    q_range,
    p_ellipse,
    cmap='gray',
    intensity_range='absolute',
    vmin=0,
    vmax=10.0,
)


In [ ]:
p_ellipse

In [ ]:
# The elliptical parameters are not automatically added to the calibration metadata,
# (to allow inspection of the fit to ensure it's accurate), so need to be added manually
# once a good fit is found. Like so:

bragg_peaks.calibration.set_p_ellipse(p_ellipse)

In [ ]:
# Note that the code above only adds (a,b,theta) to the calibration metadata; the origin needs to
# be calibrated separately, as we did above 

bragg_peaks.calibration

In [ ]:
# Calibrate, compute a new bragg vector map, and compare

bragg_peaks.setcal()
bvm_e = bragg_peaks.histogram(
    sampling=sampling
)

show([bvm_e, bvm_c], vmax=0.99 ,   annulus={
        'center':bvm_c.origin,
        'radii': q_range,'fill':True,'color':'r','alpha':0.3})

In [ ]:
# Define gold structure 

# set lattice parameter and Z-number
a_lat = 4.08
atom_num = 79

# set max scattering angle, in inverse Angstroms
k_max = 1.5


# Define lattice
pos = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])

# Make crystal
crystal = py4DSTEM.process.diffraction.Crystal(
    pos, 
    atom_num, 
    a_lat)

crystal.calculate_structure_factors(k_max)
crystal.plot_scattering_intensity()

In [ ]:
# calibrate
bragg_peaks.calibration.set_Q_pixel_size(pixel_size_inv_Ang)
bragg_peaks.calibration.set_Q_pixel_units('A^-1')
bragg_peaks.setcal()

crystal.plot_scattering_intensity(
    bragg_peaks = bragg_peaks,
    bragg_k_power = 2.0
)

In [ ]:
# fit pixel size to lattice
crystal.calibrate_pixel_size(
    bragg_peaks = bragg_peaks,
    bragg_k_power = 4.0,
    plot_result = True,
    set_calibration_in_place = True
)

In [ ]:
cal_dict = {}
cal_dict['reciprocal_space_pix(1/A)'] = bragg_peaks.calibration['Q_pixel_size']
cal_dict['p_ellipse'] = p_ellipse
cal_dict['nominal_camera_length(m)'] = nominal_CL

file_name_json = file_path.split('_data')[-2] + '_CL_' +str(cam_len) + 'cm.json'

with open(file_name_json, 'w') as fp:
    json.dump(cal_dict, fp)